In [9]:
#Word2vec Folder with cosine similarity for syria example
import csv, os, shutil, numpy as np, scipy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

csvfile = open('M:\masters\semester 4\web scraping\output files\TopicStats\Syria.csv', newline='') 
Rdf2vecFolderDirectory = "M:\\masters\\semester 4\\ProjectData\\SyriaSpeechesRdf2vec"
spamreader = csv.reader(csvfile, delimiter=',')

MPNames = [] #where the MP Names are saved (actual list from syria speeches folder)
MPParty =[] #where the MP party will be saved (should have same index as MPNames)
CSVStatsMpNames = [] #where the MP Names from the CSV Statistic file are saved
CSVStatsMpParty = [] #where the MP Party from the SCV Statistics file are saved
Rdf2vecFileTextList = [] #where the TagMe TextFiles are Saved

#function that returns the MPs party name and it takes as input the list with all mp names, list with their corresponding parties
# as well as the name of the MP we are looking for.
def getParty(CSVStatsMpNames, CSVStatsMpParty, name):
    for mpnames in CSVStatsMpNames:
        index = CSVStatsMpNames.index(mpnames)
        if(name.strip() in mpnames.strip()):
            mpparty = CSVStatsMpParty[index]
            return mpparty

for file in os.listdir(Rdf2vecFolderDirectory): #save the text from the syria files
    filedirectorylink = os.path.join(Rdf2vecFolderDirectory,file)
    fileText = open(filedirectorylink, encoding="utf8")
    lista = [float(x) for x in fileText.read().split(" ") if len(x)>2] #convert the strings into floats
    if(len(lista) == 200):#to make sure that no corrupted files are added with dimensions less/more than 200 as it will cause error in the cosine similarity
        Rdf2vecFileTextList.append(lista)


for fileName in os.listdir(word2vecFolderDirectory): #save the names to correspond the texts
    NameList = fileName.strip().split("_")
    MpName = ""
    if(len(NameList) >= 2):
        MpName = NameList[0] +" "+ NameList[1]
    else:
        MpName = NameList[0]
    MPNames.append(MpName)

for row in spamreader: #saving mpnames and parties from the statistics file
    mpname = row[0].strip()
    mpparty = row[1].strip()
    if(mpname not in CSVStatsMpNames):
        CSVStatsMpNames.append(mpname)
        CSVStatsMpParty.append(mpparty)
print(len(CSVStatsMpNames))
print(len(CSVStatsMpParty))

for MP in MPNames: #matchin MPnames from the folder with their parties
    index = MPNames.index(MP)
    mpparty = getParty(CSVStatsMpNames,CSVStatsMpParty, MP.strip())
    MPParty.insert(index, mpparty)
    

#tfidf = TfidfVectorizer().fit_transform(word2vecFileTextList) #no need for tfidf because they are already vectors
#cosine_similarities = linear_kernel(tfidf[0:1], tfidf).flatten()
#it gets the matrix as an input example, "[len(TagMeFileTextList)-1:len(TagMeFileTextList)]" or "[0:1]"
counter1 = 0

cosine_similarities = cosine_similarity(Rdf2vecFileTextList) #cosine similarity for the whole courpus (output:array of arrays)
print("cos sim is " , cosine_similarities) #print all the cosine similarity of each document with the other
for file in cosine_similarities:
    related_docs_indices = cosine_similarities[counter1].argsort()[:-6:-1] #first 5 related elements #takes the first array in the cosine similarity
                                                                                                     #array, then get the first 5 elements related to them
    print("rel docs is " , related_docs_indices)
    RelatedDocsParties = []
    RelatedDocMps =[]
    for doc in related_docs_indices:
        docparty = MPParty[doc]
        docMp = MPNames[doc]
        RelatedDocMps.append(docMp)
        RelatedDocsParties.append(docparty)
    
    counter1 = counter1 + 1 #counters are used to loop each file to compare it with all the others
        
    print("related doc mps: ", RelatedDocMps)    
    print("related doc parties: ", RelatedDocsParties)
    print("related doc indices :" ,related_docs_indices)
    print(len(related_docs_indices))
    #print(cosine_similarities[related_docs_indices])


116
116
cos sim is  [[ 1.          0.97201451  0.96822126 ...,  0.77402329  0.93729709
   0.77834254]
 [ 0.97201451  1.          0.97606441 ...,  0.71930708  0.94544497
   0.71485713]
 [ 0.96822126  0.97606441  1.         ...,  0.75382748  0.93045789
   0.76090266]
 ..., 
 [ 0.77402329  0.71930708  0.75382748 ...,  1.          0.80176594
   0.7329219 ]
 [ 0.93729709  0.94544497  0.93045789 ...,  0.80176594  1.          0.69747596]
 [ 0.77834254  0.71485713  0.76090266 ...,  0.7329219   0.69747596  1.        ]]
rel docs is  [  0   4  44 103 101]
related doc mps:  ['Alan Brown', 'Alison McGovern', 'Gavin Newlands', 'Peter Lilley', 'Peter Grant']
related doc parties:  ['Scottish National Party', 'Labour', 'Scottish National Party', 'Conservative', 'Scottish National Party']
related doc indices : [  0   4  44 103 101]
5
rel docs is  [ 1 75 81 61 12]
related doc mps:  ['Alan Mak', 'Kenneth Clarke', 'Liz Kendall', 'John Bercow', 'Bob Blackman']
related doc parties:  ['Conservative', 'Conserv